The purpose of this file is to load World bank indicator data, obtained in CSV format, downloading from the [World Bank Indicator portal](http://datatopics.worldbank.org/world-development-indicators/)  

The data is reformated to a pandas dataframe with following format specified: indicators are on columns and the country and year form a multi level row index

In [62]:
import pandas as pd
import json
from pandas_datareader import wb

In [63]:
data_dir = '.\\..\\..\\data\\'
#name of output pickle file
world_bank_file_out = "world_bank_bulk_data.pkl"
#Name of input bulk csv file from the World bank website
world_bank_file_input = "WDIData.csv"
wb_data = pd.read_csv(data_dir + world_bank_file_input)

In [64]:
wb_data.shape

(422136, 64)

#### Format as received from the World Bank:

In [65]:
wb_data.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.407647,82.827636,83.169227,83.587141,83.954293,84.23063,84.570425,NaN,NaN,NaN


In [66]:
#filter out innecessary columns (we will focus on years 1970 to 2018)
drop = ['Country Code','Indicator Name','1960','1961','1962','1963','1964',\
        '1965','1966','1967','1968','1969', '1970', '1971']
wb_data = wb_data.drop(drop, axis='columns')

In [67]:
#Switch around the data so that indicators are on columns and the country and year form a multi level row index
wb_data = wb_data.set_index(['Country Name', 'Indicator Code'])
wb_data = wb_data.stack()
wb_data = wb_data.unstack(['Indicator Code'])
wb_data = wb_data.sort_index()
wb_data.index.levels[1].name = 'Year'
wb_data.index.levels[0].name = 'Country'

#### Output format:

In [68]:
wb_data.head(20)

Indicator Code    EG.CFT.ACCS.ZS  EG.ELC.ACCS.ZS  EG.ELC.ACCS.RU.ZS  \
Country     Year                                                      
Afghanistan 1972             NaN             NaN                NaN   
            1973             NaN             NaN                NaN   
            1974             NaN             NaN                NaN   
            1975             NaN             NaN                NaN   
            1976             NaN             NaN                NaN   
            1977             NaN             NaN                NaN   
            1978             NaN             NaN                NaN   
            1979             NaN             NaN                NaN   
            1980             NaN             NaN                NaN   
            1981             NaN             NaN                NaN   
            1982             NaN             NaN                NaN   
            1983             NaN             NaN                NaN   
            1984             NaN             NaN                NaN   
            1985             NaN             NaN                NaN   
            1986             NaN             NaN                NaN   
            1987             NaN             NaN                NaN   
            1988             NaN             NaN                NaN   
            1989             NaN             NaN                NaN   
            1990             NaN            0.01                NaN   
            1991             NaN            0.01                NaN   

Indicator Code    EG.ELC.ACCS.UR.ZS  FX.OWN.TOTL.ZS  FX.OWN.TOTL.FE.ZS  \
Country     Year                                                         
Afghanistan 1972                NaN             NaN                NaN   
            1973                NaN             NaN                NaN   
            1974                NaN             NaN                NaN   
            1975                NaN             NaN                NaN   
            1976                NaN             NaN                NaN   
            1977                NaN             NaN                NaN   
            1978                NaN             NaN                NaN   
            1979                NaN             NaN                NaN   
            1980                NaN             NaN                NaN   
            1981                NaN             NaN                NaN   
            1982                NaN             NaN                NaN   
            1983                NaN             NaN                NaN   
            1984                NaN             NaN                NaN   
            1985                NaN             NaN                NaN   
            1986                NaN             NaN                NaN   
            1987                NaN             NaN                NaN   
            1988                NaN             NaN                NaN   
            1989                NaN             NaN                NaN   
            1990          52.036976             NaN                NaN   
            1991          53.809837             NaN                NaN   

Indicator Code    FX.OWN.TOTL.MA.ZS  FX.OWN.TOTL.OL.ZS  FX.OWN.TOTL.40.ZS  \
Country     Year                                                            
Afghanistan 1972                NaN                NaN                NaN   
            1973                NaN                NaN                NaN   
            1974                NaN                NaN                NaN   
            1975                NaN                NaN                NaN   
            1976                NaN                NaN                NaN   
            1977                NaN                NaN                NaN   
            1978                NaN                NaN                NaN   
            1979                NaN                NaN                NaN   
            1980                NaN   

#### Save to a pickle file:

In [69]:
#Write data to a pickle file
wb_data.to_pickle(data_dir + world_bank_file_out)